In [1]:
#imports and setup
import requests
client_ID = '413APP-001'
with open('.api_info','r') as f:
    client_sec = f.read()
grant_type='client_credentials'
auth_url = 'https://libraryanswers.fullerton.edu/api/1.1/oauth/token'
ticket_url = 'https://libraryanswers.fullerton.edu/api/1.1/ticket/create'

In [2]:
# Authorize
post_variables = dict(client_id='413APP-001',client_secret=client_sec,grant_type='client_credentials')

authorization = requests.post(auth_url, data = post_variables)

print(authorization.text)
creds = authorization.json()

{"access_token":"a9a46dd184ff383cc3ba904ec224ec8d4e59f1ac","expires_in":86400,"token_type":"Bearer","scope":"app_create"}


In [ ]:
#test ticket
ticket_payload = {'quid':'3105',
'pquestion':'This is a test created By DCP;ignore',
'pdetails':'This is a test created By DCP;ignore'
}

ticket_request = requests.post(ticket_url, data = ticket_payload, headers={'Authorization': 'Bearer {}'.format(creds['access_token'])})

if ticket_request.status_code == 200:
    print("ticket created", ticket_request.text)
else:
    print('Fail: ', ticket_request.status_code)